In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import splprep, splev
from scipy.spatial.distance import cdist
from scipy.interpolate import RegularGridInterpolator

# 1. Isobath tracing and construction of an isobath stack

In [ ]:
# contour-following algorithm


def followcont1(h, hcont, istart, jstart, xcyc=None, ycyc=None, ijstart=None, exact=None):
    """
    Follow a contour in a 2D array.
    
    Parameters:
    -----------
    h : 2D numpy array
        Input array (real, double, or integer)
    hcont : float
        The contour value to follow
    istart : int
        i index to start looking from
    jstart : int
        j index to start looking from
    xcyc : bool, optional
        Treat the i direction as cyclic
    ycyc : bool, optional
        Treat the j direction as cyclic
    exact : bool, optional
        If True, the supplied istart,jstart must be exactly correct
    
    Returns:
    --------
    is : numpy array
        Fractional i indices along the contour
    js : numpy array
        Fractional j indices along the contour
    ijstart : tuple, optional
        The i and j indices of the cell where the contour starts
    """
    
    lencont1 = 1000
    lencont = lencont1
    nan = np.nan
    
    # Initialize output arrays
    is_arr = np.full(lencont, nan, dtype=np.float64)
    js_arr = np.full(lencont, nan, dtype=np.float64)
    
    s = h.shape
    nx, ny = s[0], s[1]
    if len(s) != 2:
        print(f'followcont: input array should be 2D, actual dimensions = {len(s)}')
        return None, None
    
    # Calculate products for contour detection
    dxprod = (h - hcont) * (np.roll(h, -1, axis=0) - hcont)
    dyprod = (h - hcont) * (np.roll(h, -1, axis=1) - hcont)
    
    if xcyc is None:
        dxprod[-1, :] = 1.0
    if ycyc is None:
        dyprod[:, -1] = 1.0
    
    jcont = np.where((dxprod <= 0) | (dyprod <= 0))
    if len(jcont[0]) == 0 or (dxprod[istart, jstart] > 0 and dyprod[istart, jstart] > 0 and exact is not None):
        print(f'followcont: no contour found with value {hcont}')
        return None, None
    
    ijstart_indices = np.array(jcont)
    dist = np.sqrt((istart - ijstart_indices[0, :])**2 + (jstart - ijstart_indices[1, :])**2)
    jmin = np.argmin(dist)
    inow, jnow = ijstart_indices[0, jmin], ijstart_indices[1, jmin]
    ijstart = np.array([inow, jnow])
    
    ndir = 0
    iadds = np.array([+1, +1, -1, -1])
    jadds = np.array([-1, +1, +1, -1])
    
    if dxprod[inow, jnow] <= 0:
        js_arr[0] = jnow
        is_arr[0] = inow + (h[inow, jnow] - hcont) / (h[inow, jnow] - h[inow+1, jnow])
        ndir = 1
        ibl = inow
        jbl = jnow
        if h[inow, jnow] > hcont:
            ndir = 3
            ibl = inow + 1
            jbl = jnow
    else:
        js_arr[0] = jnow + (h[inow, jnow] - hcont) / (h[inow, jnow] - h[inow, jnow+1])
        is_arr[0] = inow
        ndir = 2
        ibl = inow
        jbl = jnow
        if h[inow, jnow] > hcont:
            ndir = 0
            ibl = inow
            jbl = jnow + 1
    
    nend = 0
    nback = 0
    nedges = 0
    ndir0 = ndir
    
    while True:
        iadd = iadds[ndir]
        jadd = jadds[ndir]
        inext = ibl + iadd
        jnext = jbl + jadd
        
        if inext >= nx:
            if xcyc is not None:
                inext = 0
            else:
                nend = 1
        elif inext < 0:
            if xcyc is not None:
                inext = nx - 1
            else:
                nend = 1
        
        if jnext >= ny:
            if ycyc is not None:
                jnext = 0
            else:
                nend = 1
        elif jnext < 0:
            if ycyc is not None:
                jnext = ny - 1
            else:
                nend = 1
        
        itr = inext
        jtr = jnext
        ibr = ibl + (ndir % 2) * (itr - ibl)
        jbr = jtr + (ndir % 2) * (jbl - jtr)
        itl = itr + (ndir % 2) * (ibl - itr)
        jtl = jbl + (ndir % 2) * (jtr - jbl)
        
        hbl = h[ibl, jbl]
        hbr = h[ibr, jbr]
        
        if hbl != hbr:
            frac = (hcont - hbl) / (hbr - hbl)
        else:
            frac = 0.5
        
        frac = min(frac, 1)
        frac = max(frac, 0)
        
        di = ibr - ibl
        if di > nx / 2:
            di -= nx
        if di < -nx / 2:
            di += nx
            
        dj = jbr - jbl
        if dj > ny / 2:
            dj -= ny
        if dj < -ny / 2:
            dj += ny
            
        inew = ibl + di * frac
        jnew = jbl + dj * frac
        
        nedges += 1
        if nedges > lencont:
            is_arr = np.concatenate([is_arr, np.full(lencont1, nan)])
            js_arr = np.concatenate([js_arr, np.full(lencont1, nan)])
            lencont += lencont1
        
        is_arr[nedges-1] = inew
        js_arr[nedges-1] = jnew
        
        if nend == 0:
            if h[itl, jtl] > hcont:
                ndir = (ndir + 1) % 4
            elif h[itr, jtr] > hcont:
                ibl = itl
                jbl = jtl
            elif h[ibr, jbr] > hcont:
                ndir = (ndir + 3) % 4
                ibl = itr
                jbl = jtr
            else:
                nend = 1
            
            if nedges != 1 and is_arr[0] == is_arr[nedges-1] and js_arr[0] == js_arr[nedges-1]:
                nback = 1
        
        if nend == 1 or nback == 1:
            break
    
    finite_mask = np.isfinite(is_arr)
    is_arr = is_arr[finite_mask]
    js_arr = js_arr[finite_mask]
    
    if nback != 1:
        ndir = (ndir0 + 2) % 4
        if ndir0 == 0:
            ibl = np.floor(is_arr[0]).astype(int)
            jbl = np.floor(js_arr[0]).astype(int)
        elif ndir0 == 1:
            ibl = np.ceil(is_arr[0]).astype(int)
            jbl = np.floor(js_arr[0]).astype(int)
        elif ndir0 == 2:
            ibl = np.floor(is_arr[0]).astype(int)
            jbl = np.ceil(js_arr[0]).astype(int)
        else:
            ibl = np.floor(is_arr[0]).astype(int)
            jbl = np.floor(js_arr[0]).astype(int)
        
        nend = 0
        nedges = 0
        isb = np.full(lencont1, nan)
        jsb = np.full(lencont1, nan)
        lencont = lencont1
        
        while True:
            iadd = iadds[ndir]
            jadd = jadds[ndir]
            inext = ibl + iadd
            jnext = jbl + jadd
            insav = inext
            jnsav = jnext
            
            if inext >= nx:
                if xcyc is not None:
                    inext = 0
                else:
                    nend = 1
            elif inext < 0:
                if xcyc is not None:
                    inext = nx - 1
                else:
                    nend = 1
            
            if jnext >= ny:
                if ycyc is not None:
                    jnext = 0
                else:
                    nend = 1
            elif jnext < 0:
                if ycyc is not None:
                    jnext = ny - 1
                else:
                    nend = 1
            
            itr = inext
            jtr = jnext
            ibr = ibl + (ndir % 2) * (itr - ibl)
            jbr = jtr + (ndir % 2) * (jbl - jtr)
            itl = itr + (ndir % 2) * (ibl - itr)
            jtl = jbl + (ndir % 2) * (jtr - jbl)
            
            hbl = -h[ibl, jbl]
            hbr = -h[ibr, jbr]
            
            if hbl != hbr:
                frac = (-hcont - hbl) / (hbr - hbl)
            else:
                frac = 0.5
            
            frac = min(frac, 1)
            frac = max(frac, 0)
            
            di = ibr - ibl
            if di > nx / 2:
                di -= nx
            if di < -nx / 2:
                di += nx
                
            dj = jbr - jbl
            if dj > ny / 2:
                dj -= ny
            if dj < -ny / 2:
                dj += ny
                
            inew = ibl + di * frac
            jnew = jbl + dj * frac
            
            nedges += 1
            if nedges > lencont:
                isb = np.concatenate([isb, np.full(lencont1, nan)])
                jsb = np.concatenate([jsb, np.full(lencont1, nan)])
                lencont += lencont1
            
            isb[nedges-1] = inew
            jsb[nedges-1] = jnew
            
            if nend == 0:
                if h[itl, jtl] <= hcont:
                    ndir = (ndir + 1) % 4
                elif h[itr, jtr] <= hcont:
                    ibl = itl
                    jbl = jtl
                elif h[ibr, jbr] <= hcont:
                    ndir = (ndir + 3) % 4
                    ibl = itr
                    jbl = jtr
                else:
                    nend = 1
            
            if nend == 1:
                break
        
        finite_mask = np.isfinite(isb)
        isb = isb[finite_mask]
        jsb = jsb[finite_mask]
        
        if len(isb) > 1:
            is_arr = np.concatenate([np.flip(isb[1:]), is_arr])
            js_arr = np.concatenate([np.flip(jsb[1:]), js_arr])
    
    if exact is not None and is_arr[-1] == istart and js_arr[-1] == jstart:
        is_arr = np.flip(is_arr)
        js_arr = np.flip(js_arr)
    
    if ijstart is not None:
        return is_arr, js_arr, ijstart
    else:
        return is_arr, js_arr

In [ ]:
# generate contour stack


def generate_contour_stack(h, start=1000, end=3000, interval=1, istart=None, jstart=None, xcyc=None, ycyc=None):
    """
    Generate a stack of contours for a range of contour values.
    
    Parameters:
    -----------
    h : 2D numpy array
        Input array
    start : float
        Starting contour value
    end : float
        Ending contour value
    interval : float
        Step between contour values
    istart : int, optional
        Starting i index for contour following
    jstart : int, optional
        Starting j index for contour following
    xcyc : bool, optional
        Treat i direction as cyclic
    ycyc : bool, optional
        Treat j direction as cyclic
    
    Returns:
    --------
    contour_stack : list of tuples
        Each tuple contains (contour_value, is_coords, js_coords)
    """
    
    if istart is None:
        istart = h.shape[0] // 2  # Default to center if not specified
    if jstart is None:
        jstart = h.shape[1] // 2  # Default to center if not specified
    
    contour_stack = []
    
    for hcont in np.arange(start, end + interval, interval):
        # Call the followcont function
        result = followcont1(h, hcont, istart, jstart, xcyc=xcyc, ycyc=ycyc)
        
        if result is not None:
            if len(result) == 2:
                is_coords, js_coords = result
            else:
                is_coords, js_coords, _ = result
            
            # Only add to stack if we got valid coordinates
            if len(is_coords) > 0 and len(js_coords) > 0:
                contour_stack.append((hcont, is_coords, js_coords))
    
    return contour_stack


# 2. Identification of cross-slope points

In [ ]:
# hybrid smoothing procedure 
# combining adaptive moving average filtering, Gaussian convolution with reflective boundaries, and spline-based curve
# reconstruction subject to arc-length preservation.

# ========== Ultra-strong contour smoothing ==========
# ========== Improved contour smoothing (preserves length) ==========
def ultra_smooth_contour_preserve_length(contour_points, sigma=2.0, spline_smooth=0.001, iterations=2):
    """
    Improved contour smoothing that preserves the original contour length
    """
    if len(contour_points) < 50:
        return contour_points
    
    # Save original endpoints
    original_start = contour_points[0].copy()
    original_end = contour_points[-1].copy()
    original_length = len(contour_points)
    
    current_points = contour_points.copy()
    
    # Use smaller sigma and fewer iterations
    for iter_num in range(iterations):
        i_coords = current_points[:, 0]
        j_coords = current_points[:, 1]
        
        # Use fixed sigma to avoid progressive over-smoothing
        current_sigma = sigma
        i_smooth = gaussian_filter1d(i_coords, sigma=current_sigma, mode='reflect')
        j_smooth = gaussian_filter1d(j_coords, sigma=current_sigma, mode='reflect')
        
        current_points = np.column_stack([i_smooth, j_smooth])
    
    try:
        t = np.linspace(0, 1, len(current_points))
        # Use a larger smoothing factor to avoid overfitting
        tck, u = splprep(
            [current_points[:, 0], current_points[:, 1]],
            s=spline_smooth * len(current_points) * 10,
            k=3
        )
        
        # Preserve the original number of points
        u_new = np.linspace(0, 1, original_length)
        i_final, j_final = splev(u_new, tck)
        
        # Light additional smoothing with a smaller sigma
        i_final = gaussian_filter1d(i_final, sigma=1.0, mode='reflect')
        j_final = gaussian_filter1d(j_final, sigma=1.0, mode='reflect')
        
        smoothed_points = np.column_stack([i_final, j_final])
        
        # Enforce original endpoints
        smoothed_points[0] = original_start
        smoothed_points[-1] = original_end
        
        return smoothed_points
        
    except Exception:
        # If spline fitting fails, return Gaussian-smoothed result with preserved endpoints
        current_points[0] = original_start
        current_points[-1] = original_end
        return current_points


def ultra_smooth_contour_stack_preserve_length(contour_stack, sigma=2.0, spline_smooth=0.001):
    """
    Improved contour stack smoothing that preserves the length of each contour
    """
    smoothed_stack = []
    
    for depth, i_coords, j_coords in contour_stack:
        points = np.column_stack([i_coords, j_coords])
        
        if len(points) > 80:
            # Use the improved smoothing method
            smoothed_points = ultra_smooth_contour_preserve_length(points, sigma, spline_smooth)
        elif len(points) > 30:
            # Use Gaussian filtering with reflective boundary conditions
            i_smooth = gaussian_filter1d(i_coords, sigma=3.0, mode='reflect')
            j_smooth = gaussian_filter1d(j_coords, sigma=3.0, mode='reflect')
            smoothed_points = np.column_stack([i_smooth, j_smooth])
        else:
            # Apply very light smoothing for short contours
            i_smooth = gaussian_filter1d(i_coords, sigma=1.5, mode='reflect')
            j_smooth = gaussian_filter1d(j_coords, sigma=1.5, mode='reflect')
            smoothed_points = np.column_stack([i_smooth, j_smooth])
        
        smoothed_stack.append((depth, smoothed_points[:, 0], smoothed_points[:, 1]))
    
    return smoothed_stack


# ========== Alternative approach: moving-average-based smoothing ==========
def moving_average_smooth_contour(contour_points, window_size=5):
    """
    Smooth contours using a moving average, better preserving contour length
    """
    if len(contour_points) < window_size * 2:
        return contour_points
    
    # Save original endpoints
    original_start = contour_points[0].copy()
    original_end = contour_points[-1].copy()
    
    i_coords = contour_points[:, 0]
    j_coords = contour_points[:, 1]
    
    # Create moving average kernel
    kernel = np.ones(window_size) / window_size
    
    # Apply moving average
    i_smooth = np.convolve(i_coords, kernel, mode='same')
    j_smooth = np.convolve(j_coords, kernel, mode='same')
    
    # Handle edge effects
    half_window = window_size // 2
    i_smooth[:half_window] = i_coords[:half_window]
    i_smooth[-half_window:] = i_coords[-half_window:]
    j_smooth[:half_window] = j_coords[:half_window]
    j_smooth[-half_window:] = j_coords[-half_window:]
    
    smoothed_points = np.column_stack([i_smooth, j_smooth])
    
    # Preserve original endpoints
    smoothed_points[0] = original_start
    smoothed_points[-1] = original_end
    
    return smoothed_points


def moving_average_smooth_stack(contour_stack, window_size=5):
    """
    Smooth a stack of contours using a moving average
    """
    smoothed_stack = []
    
    for depth, i_coords, j_coords in contour_stack:
        points = np.column_stack([i_coords, j_coords])
        
        # Adjust window size based on contour length
        adaptive_window = min(window_size, len(points) // 10)
        if adaptive_window < 3:
            adaptive_window = 3
            
        smoothed_points = moving_average_smooth_contour(points, adaptive_window)
        smoothed_stack.append((depth, smoothed_points[:, 0], smoothed_points[:, 1]))
    
    return smoothed_stack


# ========== Best approach: hybrid smoothing ==========
def hybrid_smooth_contour(contour_points, sigma=1.5, window_size=5, spline_smooth=0.01):
    """
    Hybrid smoothing method: moving average + light Gaussian filtering + conservative spline
    """
    if len(contour_points) < 50:
        return contour_points
    
    # Save original endpoints
    original_start = contour_points[0].copy()
    original_end = contour_points[-1].copy()
    original_length = len(contour_points)
    
    # Step 1: moving average smoothing
    points_ma = moving_average_smooth_contour(contour_points, window_size)
    
    # Step 2: light Gaussian filtering
    i_coords = points_ma[:, 0]
    j_coords = points_ma[:, 1]
    i_gaussian = gaussian_filter1d(i_coords, sigma=sigma, mode='reflect')
    j_gaussian = gaussian_filter1d(j_coords, sigma=sigma, mode='reflect')
    
    points_gaussian = np.column_stack([i_gaussian, j_gaussian])
    
    try:
        # Step 3: conservative spline interpolation
        t = np.linspace(0, 1, len(points_gaussian))
        tck, u = splprep(
            [points_gaussian[:, 0], points_gaussian[:, 1]],
            s=spline_smooth * len(points_gaussian) * 5,
            k=3
        )
        
        # Preserve the original number of points
        u_new = np.linspace(0, 1, original_length)
        i_final, j_final = splev(u_new, tck)
        
        final_points = np.column_stack([i_final, j_final])
        
        # Enforce original endpoints
        final_points[0] = original_start
        final_points[-1] = original_end
        
        return final_points
        
    except Exception:
        # If spline fitting fails, return Gaussian-smoothed result
        points_gaussian[0] = original_start
        points_gaussian[-1] = original_end
        return points_gaussian


def hybrid_smooth_contour_stack(contour_stack, sigma=1.5, window_size=5, spline_smooth=0.01):
    """
    Apply hybrid smoothing to a stack of contours
    """
    smoothed_stack = []
    
    for depth, i_coords, j_coords in contour_stack:
        points = np.column_stack([i_coords, j_coords])
        
        if len(points) > 60:
            smoothed_points = hybrid_smooth_contour(points, sigma, window_size, spline_smooth)
        else:
            # Use moving average for short contours
            smoothed_points = moving_average_smooth_contour(points, min(5, len(points) // 10))
        
        smoothed_stack.append((depth, smoothed_points[:, 0], smoothed_points[:, 1]))
    
    return smoothed_stack


# ========== Apply ultra-strong smoothing ==========
smoothed_selected_contours = hybrid_smooth_contour_stack(
    selected_contours, sigma=25.0, spline_smooth=0.0005
)


In [ ]:
# Identification of cross-slope points on the smoothed contours

# ========== Large-scale elliptical neighborhood normal averaging ==========
def calculate_elliptical_normal_large(contour_points, point_index, 
                                      ellipse_ratio=0.15,
                                      num_neighbors=40,
                                      max_search_ratio=0.4):
    if len(contour_points) < 15:
        return calculate_normal_at_point_basic(contour_points, point_index)
    
    # Initial local normal
    initial_normal = calculate_normal_at_point_basic(contour_points, point_index)
    if np.linalg.norm(initial_normal) < 0.1:
        return initial_normal
    
    current_point = contour_points[point_index]
    contour_length = len(contour_points)
    
    # Vectors and distances from the reference point
    vectors = contour_points - current_point
    distances = np.linalg.norm(vectors, axis=1)
    
    # Compute angular separation relative to the initial normal
    angles = []
    for vec in vectors:
        if np.linalg.norm(vec) > 0.1:
            vec_norm = vec / np.linalg.norm(vec)
            angle = np.arccos(np.clip(np.dot(vec_norm, initial_normal), -1.0, 1.0))
            angles.append(min(angle, np.pi - angle))
        else:
            angles.append(0)
    
    angles = np.array(angles)
    
    # Elliptical distance metric (elongated along the normal direction)
    ellipse_distances = np.sqrt(
        (distances * np.cos(angles))**2 / ellipse_ratio + 
        (distances * np.sin(angles))**2 * ellipse_ratio
    )
    
    # Select candidate neighbors
    max_neighbors = min(num_neighbors * 2, len(contour_points))
    candidate_indices = np.argsort(ellipse_distances)[:max_neighbors]
    
    # Further restrict neighbors by distance percentile
    max_distance = np.percentile(distances[candidate_indices], 70)
    valid_candidates = [
        idx for idx in candidate_indices
        if distances[idx] <= max_distance and idx != point_index
    ]
    
    # Fallback if too few valid neighbors
    if len(valid_candidates) < 10:
        valid_candidates = candidate_indices[:min(20, len(candidate_indices))]
        valid_candidates = [idx for idx in valid_candidates if idx != point_index]
    
    # Compute weighted average of neighboring normals
    neighbor_normals = []
    neighbor_weights = []
    
    for idx in valid_candidates:
        neighbor_normal = calculate_normal_at_point_basic(contour_points, idx)
        
        if np.linalg.norm(neighbor_normal) > 0.1:
            weight = 1.0 / (1.0 + distances[idx])
            neighbor_normals.append(neighbor_normal)
            neighbor_weights.append(weight)
    
    if len(neighbor_normals) == 0:
        return initial_normal
    
    neighbor_normals_array = np.array(neighbor_normals)
    neighbor_weights_array = np.array(neighbor_weights)
    neighbor_weights_array /= np.sum(neighbor_weights_array)
    
    avg_normal = np.zeros(2)
    for i in range(len(neighbor_normals)):
        avg_normal += neighbor_normals_array[i] * neighbor_weights_array[i]
    
    avg_norm = np.linalg.norm(avg_normal)
    
    # Normalize and ensure consistent orientation
    if avg_norm > 0.1:
        avg_normal /= avg_norm
        if np.dot(avg_normal, initial_normal) < 0:
            avg_normal = -avg_normal
        return avg_normal
    else:
        return initial_normal


# ========== Basic local normal estimation ==========
def calculate_normal_at_point_basic(contour_points, point_index, window_size=9):
    if len(contour_points) < 3:
        return np.array([0, 0])
    
    start_idx = max(0, point_index - window_size)
    end_idx = min(len(contour_points), point_index + window_size + 1)
    
    if end_idx - start_idx < 3:
        return np.array([0, 0])
    
    window_points = contour_points[start_idx:end_idx]
    t = np.arange(len(window_points))
    
    try:
        # Linear fit for i-coordinate
        A_i = np.column_stack([t, np.ones(len(t))])
        coeff_i, _, _, _ = np.linalg.lstsq(A_i, window_points[:, 0], rcond=None)
        
        # Linear fit for j-coordinate
        A_j = np.column_stack([t, np.ones(len(t))])
        coeff_j, _, _, _ = np.linalg.lstsq(A_j, window_points[:, 1], rcond=None)
        
        # Tangent vector
        tangent = np.array([coeff_i[0], coeff_j[0]])
        tangent_norm = np.linalg.norm(tangent)
        
        if tangent_norm > 0:
            tangent /= tangent_norm
            normal = np.array([-tangent[1], tangent[0]])
            return normal
        else:
            return np.array([0, 0])
    except Exception:
        return np.array([0, 0])


# ========== Select reference points on each contour and compute large-scale averaged normals ==========
def select_points_with_large_elliptical_normals(contour_stack, points_per_contour=6):
    reference_points = []
    reference_normals = []
    
    for depth_idx, (depth, i_coords, j_coords) in enumerate(contour_stack):
        contour_points = np.column_stack([i_coords, j_coords])
        
        # Determine sampling indices along the contour
        if len(contour_points) < points_per_contour * 2:
            indices = np.arange(len(contour_points))
        else:
            start_idx = len(contour_points) // 55
            end_idx = len(contour_points) - len(contour_points) // 15
            indices = np.linspace(start_idx, end_idx - 1, points_per_contour, dtype=int)
        
        points_on_contour = []
        normals_on_contour = []
        
        for idx in indices:
            point = contour_points[idx]
            normal = calculate_elliptical_normal_large(
                contour_points, idx,
                ellipse_ratio=0.15,
                num_neighbors=50,
                max_search_ratio=0.5
            )
            
            if np.linalg.norm(normal) > 0.2:
                points_on_contour.append(point)
                normals_on_contour.append(normal)
        
        reference_points.append((depth, points_on_contour))
        reference_normals.append((depth, normals_on_contour))
    
    return reference_points, reference_normals


# ========== Select reference points and compute large-scale averaged normals ==========
points_per_contour = 20 
reference_points, reference_normals = select_points_with_large_elliptical_normals(
    smoothed_selected_contours, points_per_contour
)


# 3. Extraction of bottom pressure

In [ ]:
# ========== Helper function: convert fractional coordinates to actual coordinates ==========
def fractional_to_actual(lon_array, lat_array, y_frac, x_frac):
    """
    Convert fractional grid coordinates to actual longitude and latitude
    
    Parameters:
        lon_array: 2D longitude array (y, x)
        lat_array: 2D latitude array (y, x)
        y_frac: fractional y-coordinates
        x_frac: fractional x-coordinates
    
    Returns:
        lon_pts: interpolated longitude values
        lat_pts: interpolated latitude values
    """
    # Clip coordinates to valid grid range
    y_coords = np.clip(y_frac, 0, lon_array.shape[0] - 1)
    x_coords = np.clip(x_frac, 0, lon_array.shape[1] - 1)
    
    # Bilinear interpolation to obtain longitude and latitude
    y_floor = np.floor(y_coords).astype(int)
    y_ceil = np.ceil(y_coords).astype(int)
    x_floor = np.floor(x_coords).astype(int)
    x_ceil = np.ceil(x_coords).astype(int)
    
    # Prevent index out of bounds
    y_ceil = np.minimum(y_ceil, lon_array.shape[0] - 1)
    x_ceil = np.minimum(x_ceil, lon_array.shape[1] - 1)
    
    # Interpolation weights
    y_weight = y_coords - y_floor
    x_weight = x_coords - x_floor
    
    # Bilinear interpolation
    lon_pts = (
        (1 - y_weight) * (1 - x_weight) * lon_array[y_floor, x_floor]
        + (1 - y_weight) * x_weight * lon_array[y_floor, x_ceil]
        + y_weight * (1 - x_weight) * lon_array[y_ceil, x_floor]
        + y_weight * x_weight * lon_array[y_ceil, x_ceil]
    )
    
    lat_pts = (
        (1 - y_weight) * (1 - x_weight) * lat_array[y_floor, x_floor]
        + (1 - y_weight) * x_weight * lat_array[y_floor, x_ceil]
        + y_weight * (1 - x_weight) * lat_array[y_ceil, x_floor]
        + y_weight * x_weight * lat_array[y_ceil, x_ceil]
    )
    
    return lon_pts, lat_pts


In [ ]:
# ========== Extract OBP at reference points ==========
def extract_pressure_at_reference_points(lon_sub, lat_sub, pbo_sub, reference_points):
    """
    Extract pressure values at reference point locations
    
    Parameters:
        lon_sub: 2D longitude array (y, x)
        lat_sub: 2D latitude array (y, x)
        pbo_sub: 3D pressure array (time, y, x)
        reference_points: list of reference points
                          [(depth, [point1, point2, ...]), ...]
    
    Returns:
        pbo_ds: xarray Dataset
    """
    # Get depth list
    depths = [depth for depth, _ in reference_points]
    max_points = max(len(points) for _, points in reference_points)
    t_steps = pbo_sub.shape[0]
    
    # Initialize output arrays
    pressure_data = np.full((t_steps, len(depths), max_points), np.nan)
    lon_data = np.full((len(depths), max_points), np.nan)
    lat_data = np.full((len(depths), max_points), np.nan)
    
    print(
        f"Start extracting OBP at reference points: "
        f"{len(depths)} depth levels, up to {max_points} points per depth, "
        f"{t_steps} time steps"
    )
    
    # Loop over reference points at each depth
    for depth_idx, (depth, points) in enumerate(reference_points):
        if len(points) == 0:
            continue
            
        n_points = len(points)
        
        # Extract y, x coordinates
        y_coords = np.array([point[0] for point in points])
        x_coords = np.array([point[1] for point in points])
        
        # Convert to actual longitude and latitude
        lon_pts, lat_pts = fractional_to_actual(
            lon_sub, lat_sub, y_coords, x_coords
        )
        
        # Store coordinates
        lon_data[depth_idx, :n_points] = lon_pts
        lat_data[depth_idx, :n_points] = lat_pts
        
        # Prepare interpolation points (ensure valid range)
        points_array = np.column_stack([
            np.clip(y_coords, 0, lon_sub.shape[0] - 1.0001),
            np.clip(x_coords, 0, lon_sub.shape[1] - 1.0001)
        ])
        
        # Interpolate pressure for each time step
        for t in range(t_steps):
            interp = RegularGridInterpolator(
                (np.arange(lon_sub.shape[0]), np.arange(lon_sub.shape[1])),
                pbo_sub[t],
                method="linear",
                bounds_error=False,
                fill_value=np.nan
            )
            pressure_data[t, depth_idx, :n_points] = interp(points_array)
        
        print(f"Depth {depth} m: processed {n_points} reference points")
    
    # Create xarray Dataset
    ds = xr.Dataset(
        {
            "pressure": (["time", "depth", "point"], pressure_data),
            "lon": (["depth", "point"], lon_data),
            "lat": (["depth", "point"], lat_data),
        },
        coords={
            "time": np.arange(t_steps),
            "depth": depths,
            "point": np.arange(max_points),
        }
    )
    
    return obp_ds


def extract_nth_point_obp(obp_ds, n=0):
    """
    Extract OBP time series at the n-th point (point = n) as a (time, depth) array
    
    Parameters:
        obp_ds: xarray Dataset
        n: point index (starting from 0)
    """
    # Check index validity
    if n >= obp_ds.dims["point"]:
        print(
            f"Error: index {n} is out of range, "
            f"maximum index is {obp_ds.dims['point'] - 1}"
        )
        return None
    
    # Extract pressure at the n-th point
    nth_point_obp = obp_ds.pressure.isel(point=n).values  # shape: (time, depth)
    
    print(f"OBP extraction for point {n} completed!")
    print(f"Array shape: {nth_point_obp.shape} (time × depth)")
    print(f"Number of time steps: {nth_point_obp.shape[0]}")
    print(f"Number of depth levels: {nth_point_obp.shape[1]}")
    
    # Basic statistics
    valid_mask = ~np.isnan(nth_point_obp)
    print(f"Valid data points: {np.sum(valid_mask)}/{nth_point_obp.size}")
    print(f"Data completeness: {np.sum(valid_mask) / nth_point_obp.size * 100:.2f}%")
    
    if np.any(valid_mask):
        print(
            f"OBP range: {np.nanmin(nth_point_obp):.3f} "
            f"to {np.nanmax(nth_point_obp):.3f} Pa"
        )
        print(f"Mean OBP: {np.nanmean(nth_point_obp):.3f} Pa")
        print(f"OBP standard deviation: {np.nanstd(nth_point_obp):.3f} Pa")
    
    return nth_point_obp


def calculate_nth_point_mean_latitude(obp_ds, n=0):
    """
    Calculate the mean latitude of the n-th point (point = n)
    
    Parameters:
        obp_ds: xarray Dataset
        n: point index (starting from 0)
    """
    # Check index validity
    if n >= obp_ds.dims["point"]:
        print(
            f"Error: index {n} is out of range, "
            f"maximum index is {obp_ds.dims['point'] - 1}"
        )
        return np.nan, None
    
    # Extract latitude at the n-th point
    nth_point_lat = obp_ds.lat.isel(point=n).values  # shape: (depth,)
    
    print(f"Latitude data for point {n}:")
    print(f"Latitude array shape: {nth_point_lat.shape}")
    print(
        f"Latitude range: {np.nanmin(nth_point_lat):.3f}°N "
        f"to {np.nanmax(nth_point_lat):.3f}°N"
    )
    
    # Compute mean latitude (ignore NaNs)
    valid_lat_mask = ~np.isnan(nth_point_lat)
    valid_lat_count = np.sum(valid_lat_mask)
    
    if valid_lat_count > 0:
        mean_latitude = np.nanmean(nth_point_lat)
        lat_std = np.nanstd(nth_point_lat)
        lat_min = np.nanmin(nth_point_lat)
        lat_max = np.nanmax(nth_point_lat)
        
        print(f"\nLatitude statistics for point {n}:")
        print(f"  Mean latitude: {mean_latitude:.3f}°N")
        print(f"  Latitude standard deviation: {lat_std:.3f}°")
        print(f"  Latitude range: {lat_min:.3f}°N to {lat_max:.3f}°N")
        print(
            f"  Valid depth levels: {valid_lat_count}/{len(nth_point_lat)}"
        )
        print(
            f"  Data completeness: "
            f"{valid_lat_count / len(nth_point_lat) * 100:.2f}%"
        )
        
        # Show latitude values for the first 10 depth levels
        print("\nLatitude values for the first 10 depth levels:")
        for i in range(min(10, len(nth_point_lat))):
            depth = obp_ds.depth.values[i]
            lat = nth_point_lat[i]
            if not np.isnan(lat):
                print(f"  Depth {depth:.0f} m: {lat:.3f}°N")
            else:
                print(f"  Depth {depth:.0f} m: NaN")
                
    else:
        print(f"Warning: no valid latitude data for point {n}")
        mean_latitude = np.nan
    
    return mean_latitude, nth_point_lat
